In [1]:
# Kyle Lund
# CS-340

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CRUD

from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU123"
host = 'nv-desktop-services.apporto.com'
port = 27017
database = "AAC"
collection = "animals"

db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

# DEBUG
#print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                         style={"height": "10%", "width": "10%"})),
    html.Center(html.B(html.H1("Kyle Lund's CS-340 Dashboard"))),
    html.Hr(),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
        
    ),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(db.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(db.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(db.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(db.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:25351/
